# Modified to create an animation showing analysis method -- Event finder & raw data checker

This notebook will find competitive cellular "events" in the simplest definition (i.e. a loser cell apoptosis) and return information about the spatiotemporal distrubition of counterpart competitive events (i.e. winner cell mitosis)

Contents:

- Load modules
- Load cell finding functions
- Set experiment data path
- Load image data
- Load tracking data
- Apply necessary coordinate shift for viewer
- Isolate one track of interest (target track)
- Find corresponding tracks/events of interest within a given spatiotemporal range (E.g. if target track is Scr apoptosis, then find all nearby wild-type mitosis)
- Set points and regions of interest for highlighting in Napari viewer(now set as functions so this bit is automatic
- Launch Napari image viewer if desired


In [1]:
import napari
import btrack
import numpy as np
from skimage.io import imread
import os
print("Napari version no.:", napari.__version__)
print("btrack version no.:", btrack.__version__)
from btrack.utils import import_HDF, import_JSON, tracks_to_napari
from tqdm.notebook import tnrange, tqdm
import matplotlib.pyplot as plt
import tools

Napari version no.: 0.4.7
btrack version no.: 0.4.0


# functions to find local neighbourhood info

In [2]:
def euclidean_distance(target_track, other_track, frame):
    try:
        idx0 = target_track.t.index(frame) #target_track.t.index(find_apoptosis_time(target_track, index = False)) ## could also do just ## apop_index
        idx1 = other_track.t.index(frame)
    except:
        return np.inf
    
    dx = target_track.x[idx0] - other_track.x[idx1]
    dy = target_track.y[idx0] - other_track.y[idx1]
    
    return np.sqrt(dx**2 + dy**2)

def find_apoptosis_time(target_track, index): ### if index is set to True then the index of the apoptotic time (wrt target_track) is returned
    for i, j in enumerate(target_track.label):
        if j == 'APOPTOSIS' and target_track.label[i+1] == 'APOPTOSIS' and target_track.label[i+2] == 'APOPTOSIS': # and target_track.label[i+3] =='APOPTOSIS' and target_track.label[i+4] =='APOPTOSIS':
            apop_index = i
            break
    apop_time = target_track.t[apop_index]
    if index == True: 
        return apop_index
    else: 
        return apop_time

def find_nearby_wt_mitosis(target_track, delta_t, radius):
    frame = find_apoptosis_time(target_track, index = False) + delta_t 
    dividing_states = ('METAPHASE',) #('PROMETAPHASE', 'METAPHASE', 'DIVIDE')
    wt_tracks_in_radius = [wt_track for wt_track in wt_tracks if wt_track.in_frame(frame) if euclidean_distance(target_track, wt_track, frame)<radius]
    wt_mitosis_in_radius = [wt_track for wt_track in wt_tracks if wt_track.in_frame(frame) if euclidean_distance(target_track, wt_track, frame)<radius if wt_track.label[wt_track.t.index(frame)] in dividing_states if wt_track.fate.name == "DIVIDE"] ###check this
  
    return wt_tracks_in_radius, wt_mitosis_in_radius


def maximum_xyt(i):
    #### BEWARE xy coord switch!     
    if i == 'x':
        max_x = int((max([max(i.y) for i in wt_tracks])))
        return max_x
    if i == 'y':
        max_y = int((max([max(i.x) for i in wt_tracks])))
        return max_y
    if i == 't':
        max_t = int((max([max(i.t) for i in wt_tracks])))
        return max_t

### Set experiment data path 

In [3]:
# print("Input experiment number")
# experiment_no = input()
# root_path = os.path.join('/home/nathan/data/kraken/h2b/giulia/', experiment_no)
root_path = '/home/nathan/data/kraken/h2b/giulia/GV0807'  ## this overwrites input option for ease 
gfp_path = os.path.join(root_path, 'Pos3/stacks/gfp.tif')
rfp_path = os.path.join(root_path, 'Pos3/stacks/rfp.tif')
bf_path = os.path.join(root_path, 'Pos3/stacks/bf.tif')
tracks_path = os.path.join(root_path, 'Pos3/Pos3_aligned/HDF/segmented.hdf5')

### Load image data

In [4]:
gfp = imread(gfp_path)

In [5]:
rfp = imread(rfp_path)

In [ ]:
bf = imread(bf_path) #3 large 1000+ frame stacks is usually too much for memory to process on 16gb ram

### Load tracking data

In [29]:
with btrack.dataio.HDF5FileHandler(tracks_path, 'r', obj_type = "obj_type_1") as hdf:
    wt_tracks = hdf.tracks
with btrack.dataio.HDF5FileHandler(tracks_path, 'r', obj_type = "obj_type_2") as hdf:
    scr_tracks = hdf.tracks

## this method casues problems as the viewer for tracks doesnt like negative numbers
#wt_tracks, scr_tracks, all_tracks = tools.load_tracking_data(tracks_path)

print("Tracks loaded")

### finding coord range of aligned images, coords switched already ## need to sort out the order of try excepts
try:
    align_x_range, align_y_range = gfp.shape[2], gfp.shape[1]    
except:
    print()
    try:
        align_x_range, align_y_range = bf.shape[2], bf.shape[1]
    except:
        print("Error: no image data loaded to map tracks to")
try: 
    align_x_range, align_y_range = rfp.shape[2], rfp.shape[1]
except:
    print()
    try:
        align_x_range, align_y_range = bf.shape[2], bf.shape[1]
    except:
        print("Error: no image data loaded to map tracks to")


### finding maximum extent of tracking coords
tracks_x_range = round(max([max(track.x) for track in wt_tracks]))
tracks_y_range = round(max([max(track.y) for track in wt_tracks])) + 2 ## sort this lazy hack out later

### coord switch
tmp = tracks_y_range
tracks_y_range = tracks_x_range
tracks_x_range = tmp

print("tracks range:", (tracks_x_range), (tracks_y_range))
print("aligned image range:", (align_x_range), (align_y_range))

shift_x = int((align_x_range - tracks_x_range)/2)
shift_y = int((align_y_range - tracks_y_range)/2)

print("shift in x and y:", shift_x, shift_y)

wt_data, properties, graph = tracks_to_napari(wt_tracks, ndim = 2)
scr_data, properties, graph = tracks_to_napari(scr_tracks, ndim = 2)

tmp = wt_data[:,2].copy() ## copy the true_y coord
wt_data[:,2] = wt_data[:,3]  ##assign the old_y coord as the true_x
wt_data[:,3] = tmp ## assign the old_x as true_y

wt_data[:,2] += shift_y ## TRUE_Y (vertical axis)
wt_data[:,3] += shift_x ## TRUE_X (horizontal axis)

tmp = scr_data[:,2].copy()
scr_data[:,2] = scr_data[:,3]
scr_data[:,3] = tmp

scr_data[:,2] += shift_y ## TRUE_Y (vertical axis)
scr_data[:,3] += shift_x ## TRUE_X (horizontal axis)

print("coordinate shift applied")

[INFO][2021/04/21 01:41:34 pm] Opening HDF file: /home/nathan/data/kraken/h2b/giulia/GV0807/Pos3/Pos3_aligned/HDF/segmented.hdf5...
[INFO][2021/04/21 01:41:34 pm] Loading tracks/obj_type_1
[INFO][2021/04/21 01:41:37 pm] Loading objects/obj_type_1 (408973, 5) (388394 filtered: area>=100)
[INFO][2021/04/21 01:41:41 pm] Closing HDF file: /home/nathan/data/kraken/h2b/giulia/GV0807/Pos3/Pos3_aligned/HDF/segmented.hdf5
[INFO][2021/04/21 01:41:41 pm] Opening HDF file: /home/nathan/data/kraken/h2b/giulia/GV0807/Pos3/Pos3_aligned/HDF/segmented.hdf5...
[INFO][2021/04/21 01:41:41 pm] Loading tracks/obj_type_2
[INFO][2021/04/21 01:41:41 pm] Loading objects/obj_type_2 (12115, 5) (8894 filtered: area>=100)
[INFO][2021/04/21 01:41:41 pm] Closing HDF file: /home/nathan/data/kraken/h2b/giulia/GV0807/Pos3/Pos3_aligned/HDF/segmented.hdf5


Tracks loaded
tracks range: 1600 1200
aligned image range: 1739 1377
shift in x and y: 69 88
coordinate shift applied


### Isolate one scribble track of interest

In [7]:
## isolate one target scribble track of interest
print("input scribble track of interest ID")
cell_ID = 17#int(input())
target_track = [scr_track for scr_track in scr_tracks if scr_track.ID == cell_ID][0]
apop_time, apop_index = find_apoptosis_time(target_track, index = False), find_apoptosis_time(target_track, index = True)  

input scribble track of interest ID


In [19]:
target_track

,ID,t,x,y,z,parent,root,state,generation,dummy
0,-17,354,592.089050,1121.349365,0.000000,2,2,3,1,False
1,-17,355,599.770813,1122.750000,0.000000,2,2,3,1,False
2,-17,356,597.294983,1122.762573,0.000000,2,2,3,1,False
3,-17,357,595.140930,1121.939575,0.000000,2,2,3,1,False
4,-17,358,596.659119,1121.590942,0.000000,2,2,3,1,False
5,-17,359,598.070312,1123.187500,0.000000,2,2,3,1,False
6,-17,360,601.812317,1124.481201,0.000000,2,2,3,1,False
7,-17,361,606.079834,1126.327026,0.000000,2,2,3,1,False
8,-17,362,608.422485,1126.737915,0.000000,2,2,0,1,False
9,-17,363,610.502502,1127.165894,0.000000,2,2,0,1,False


### Isolate and highlight (in viewer) a WT track of interest

In [13]:
wt_ID = 300
wt_target_track = [wt_track for wt_track in wt_tracks if wt_track.ID == wt_ID][0]

# Find cells of interest
and display ID for all tracks found in radius at specific delta_t frame

In [8]:
radius, inner_radius = 800,0#120, 100
delta_t = 0 ### determines which frame to open the viewer
#t_range = 20###????
wt_tracks_in_radius, wt_mitosis_in_radius = find_nearby_wt_mitosis(target_track, delta_t, radius)
print("ID for all tracks in radius:", [j.ID for i, j in enumerate(wt_tracks_in_radius)])
print("ID for all mitoses in radius:", [j.ID for i, j in enumerate(wt_mitosis_in_radius)])

ID for all tracks in radius: [42, 4, 94, 109, 143, 169, 183, 185, 189, 218, 295, 311, 330, 340, 346, 344, 380, 381, 397, 398, 405, 412, 420, 421, 428, 441, 437, 436, 445, 446, 448, 455, 456, 462, 472, 468, 490, 493, 495, 494, 496, 497, 513, 512, 508, 507, 526, 527, 528, 529, 533, 535, 534, 541, 542, 543, 545, 554, 555, 561, 560, 563, 568, 584, 585, 587, 588, 589, 591, 593, 594, 600, 601, 602, 603, 604, 605, 606, 609, 612, 613, 615, 618, 621, 622, 628, 629, 631, 632, 634, 635, 642, 644, 647, 648, 649, 650, 657, 658, 664, 666, 673, 674, 678, 683, 688, 689, 690, 691, 703, 704, 705, 706, 707, 708, 712, 719, 717, 720, 721, 728, 732, 733, 734, 736, 737, 738, 744, 745, 748, 751, 752, 753, 755, 756, 757, 763, 762, 760, 761, 765, 768, 769, 770, 771, 774, 776, 781, 785, 787, 788, 795, 797, 798, 799, 800, 803, 810, 811, 805, 813, 814, 818, 822, 823, 825, 826, 836, 837, 840, 842, 844, 845, 848, 849, 850, 851, 854, 856, 855, 853, 857, 858, 859, 861, 863, 864, 865, 867, 874, 876, 878, 879, 882, 893,

display ID for all tracks in radius and over **_delta_t range_** (can take a while)

In [9]:
import timeit
start = timeit.default_timer()
radius = 200
inner_radius = 0
t_range = 200 ###????
for delta_t in range(-abs(t_range/2), +abs(t_range/2)+1):
    wt_tracks_in_radius, wt_mitosis_in_radius = find_nearby_wt_mitosis(target_track, delta_t, radius)
    if len(wt_tracks_in_radius) > 0:
        print("Frame:",find_apoptosis_time(target_track, index = False) + delta_t)
        print("ID for all tracks in radius:", [j.ID for i, j in enumerate(wt_tracks_in_radius)])
        print("ID for all mitoses in radius:", [j.ID for i, j in enumerate(wt_mitosis_in_radius)])
stop = timeit.default_timer()
print('Any cell counter calculation time: ', round((stop - start),2), 'seconds') 

Frame: 530
ID for all tracks in radius: [148, 157, 239, 287, 308, 309, 315, 316, 317, 318, 335, 336, 340, 345, 352, 353, 367, 368, 380, 381, 389, 399, 403, 405, 407, 436, 453, 454]
ID for all mitoses in radius: []
Frame: 531
ID for all tracks in radius: [148, 157, 239, 287, 308, 309, 315, 316, 317, 318, 335, 336, 340, 345, 352, 353, 367, 368, 380, 381, 389, 399, 403, 405, 407, 436, 453, 454]
ID for all mitoses in radius: []
Frame: 532
ID for all tracks in radius: [148, 157, 239, 287, 308, 309, 315, 316, 317, 318, 335, 336, 352, 353, 367, 368, 380, 381, 389, 399, 403, 405, 407, 453, 454]
ID for all mitoses in radius: []
Frame: 533
ID for all tracks in radius: [148, 157, 239, 308, 309, 315, 316, 317, 318, 335, 336, 352, 353, 367, 368, 380, 381, 389, 399, 403, 405, 407, 453, 454]
ID for all mitoses in radius: []
Frame: 534
ID for all tracks in radius: [148, 157, 239, 308, 309, 315, 316, 317, 318, 335, 336, 352, 353, 367, 368, 380, 381, 389, 399, 403, 405, 407, 453, 454]
ID for all mitoses

Frame: 571
ID for all tracks in radius: [157, 308, 309, 315, 316, 318, 335, 336, 340, 339, 345, 352, 353, 367, 368, 380, 381, 389, 399, 403, 405, 407, 453, 454, 535, 587, 589, 591]
ID for all mitoses in radius: []
Frame: 572
ID for all tracks in radius: [308, 309, 315, 316, 318, 335, 336, 340, 339, 345, 352, 353, 367, 368, 380, 381, 389, 399, 403, 405, 407, 436, 453, 454, 535, 587, 589, 591]
ID for all mitoses in radius: []
Frame: 573
ID for all tracks in radius: [308, 309, 315, 316, 318, 335, 336, 340, 339, 345, 352, 353, 367, 368, 380, 381, 389, 399, 403, 405, 407, 436, 453, 454, 535, 587, 588, 589, 591]
ID for all mitoses in radius: []
Frame: 574
ID for all tracks in radius: [308, 309, 315, 316, 318, 335, 336, 340, 339, 345, 352, 353, 367, 368, 380, 381, 389, 399, 403, 405, 407, 436, 453, 454, 535, 587, 588, 589, 591]
ID for all mitoses in radius: []
Frame: 575
ID for all tracks in radius: [308, 309, 315, 316, 318, 335, 336, 340, 339, 346, 345, 352, 353, 367, 368, 380, 381, 389, 399

Frame: 610
ID for all tracks in radius: [157, 316, 331, 335, 336, 352, 353, 367, 368, 380, 381, 389, 405, 407, 453, 454, 535, 560, 587, 588, 591, 603, 604, 649, 650, 657, 673, 674]
ID for all mitoses in radius: []
Frame: 611
ID for all tracks in radius: [157, 316, 331, 335, 336, 352, 353, 367, 368, 380, 381, 389, 405, 407, 453, 454, 535, 560, 587, 588, 591, 603, 604, 649, 650, 657, 673, 674]
ID for all mitoses in radius: []
Frame: 612
ID for all tracks in radius: [157, 316, 331, 335, 336, 352, 353, 367, 368, 380, 381, 389, 405, 407, 453, 454, 461, 535, 561, 560, 587, 588, 591, 603, 604, 649, 650, 657, 673, 674]
ID for all mitoses in radius: []
Frame: 613
ID for all tracks in radius: [157, 316, 331, 335, 336, 340, 352, 353, 367, 368, 380, 381, 389, 405, 407, 453, 454, 461, 535, 560, 587, 588, 591, 603, 604, 649, 650, 657, 673, 674]
ID for all mitoses in radius: []
Frame: 614
ID for all tracks in radius: [157, 316, 331, 335, 336, 340, 352, 353, 367, 368, 381, 389, 405, 407, 453, 454, 461

Frame: 648
ID for all tracks in radius: [331, 335, 336, 352, 353, 367, 381, 389, 399, 405, 407, 453, 454, 461, 535, 561, 587, 591, 604, 649, 650, 657, 673, 674, 720, 721, 751, 752]
ID for all mitoses in radius: [353, 367]
Frame: 649
ID for all tracks in radius: [331, 335, 336, 352, 353, 381, 389, 399, 405, 407, 453, 454, 461, 535, 561, 587, 591, 604, 649, 650, 657, 673, 674, 720, 721, 751, 752, 753, 757]
ID for all mitoses in radius: [353]
Frame: 650
ID for all tracks in radius: [331, 335, 336, 352, 381, 389, 399, 405, 407, 453, 454, 461, 535, 561, 587, 591, 604, 649, 650, 657, 673, 674, 720, 721, 751, 752, 753, 757, 762, 761]
ID for all mitoses in radius: []
Frame: 651
ID for all tracks in radius: [331, 335, 336, 352, 380, 381, 389, 399, 405, 407, 453, 454, 461, 535, 561, 587, 591, 604, 649, 650, 657, 673, 674, 720, 721, 751, 752, 753, 757, 762, 761]
ID for all mitoses in radius: [380, 751]
Frame: 652
ID for all tracks in radius: [331, 335, 336, 352, 380, 381, 389, 399, 405, 407, 453,

Frame: 682
ID for all tracks in radius: [335, 352, 381, 399, 405, 407, 453, 535, 561, 587, 591, 604, 649, 650, 657, 673, 674, 720, 721, 751, 752, 753, 757, 762, 761, 792, 798, 803, 837, 848, 849, 865, 867]
ID for all mitoses in radius: [849]
Frame: 683
ID for all tracks in radius: [335, 352, 381, 399, 405, 407, 453, 535, 561, 587, 591, 604, 649, 650, 657, 673, 674, 720, 721, 751, 752, 753, 757, 762, 761, 792, 798, 803, 836, 837, 848, 849, 865, 867]
ID for all mitoses in radius: []
Frame: 684
ID for all tracks in radius: [335, 352, 381, 399, 405, 407, 535, 561, 587, 591, 604, 649, 650, 657, 673, 674, 720, 721, 751, 752, 753, 757, 762, 761, 792, 798, 803, 836, 837, 848, 849, 865, 867]
ID for all mitoses in radius: []
Frame: 685
ID for all tracks in radius: [335, 352, 381, 399, 405, 407, 535, 561, 587, 591, 604, 649, 650, 657, 673, 674, 720, 721, 751, 752, 753, 757, 762, 761, 792, 798, 803, 836, 837, 848, 849, 865, 867]
ID for all mitoses in radius: []
Frame: 686
ID for all tracks in radi

Frame: 716
ID for all tracks in radius: [381, 405, 535, 587, 591, 604, 649, 650, 657, 673, 674, 720, 721, 751, 752, 753, 757, 762, 761, 792, 798, 837, 848, 849, 865, 897, 900, 906, 905, 901, 924, 932, 933]
ID for all mitoses in radius: []
Frame: 717
ID for all tracks in radius: [381, 405, 535, 587, 591, 604, 649, 650, 657, 673, 674, 720, 721, 751, 752, 753, 757, 762, 761, 792, 798, 837, 848, 849, 865, 900, 906, 905, 901, 924, 932, 933]
ID for all mitoses in radius: []
Frame: 718
ID for all tracks in radius: [381, 405, 535, 587, 591, 604, 649, 650, 657, 673, 674, 720, 721, 751, 752, 753, 757, 762, 761, 791, 792, 798, 837, 848, 849, 865, 900, 906, 905, 901, 924, 932, 933]
ID for all mitoses in radius: []
Frame: 719
ID for all tracks in radius: [381, 405, 535, 587, 588, 604, 649, 650, 657, 673, 674, 720, 721, 751, 752, 753, 757, 762, 761, 792, 798, 837, 848, 849, 865, 900, 906, 905, 901, 924, 932, 933]
ID for all mitoses in radius: []
Frame: 720
ID for all tracks in radius: [381, 405, 535

### Functions to display local neighbourhood for cells of interest

In [30]:
def plot_mitoses(cell_ID, radius, delta_t): ## this function plots mitosis events into the napari viewer
    target_track = [track for track in all_tracks if track.ID == cell_ID][0]
    apop_time, apop_index = find_apoptosis_time(target_track, index = False), find_apoptosis_time(target_track, index = True)  
    apop_event = target_track.t[apop_index], target_track.x[apop_index]+shift_y, target_track.y[apop_index]+shift_x ## with transposed shift
    wt_tracks_in_radius, wt_mitosis_in_radius = find_nearby_wt_mitosis(target_track, delta_t, radius)
    t_m, x_m, y_m = np.zeros(len(wt_mitosis_in_radius)), np.zeros(len(wt_mitosis_in_radius)), np.zeros(len(wt_mitosis_in_radius))
    mito_events = np.zeros((len(wt_mitosis_in_radius), 3)) ## 3 because of the 3 cartesian coords 
    for i, wt_mitosis in enumerate(wt_mitosis_in_radius): ## this now assumes that the mitosis time point of relevance isnt the last frame of track but the time at delta_t, need to bolster definition of mitosis
        mito_index = [j for j, k in enumerate(wt_mitosis.t) if k == apop_event[0]+delta_t][0] ### [0] bc first item of list comprehension
        t_m[i], x_m[i], y_m[i] = wt_mitosis.t[mito_index], wt_mitosis.x[mito_index]+shift_y, wt_mitosis.y[mito_index]+shift_x ## plus transposed coordinate shift
        mito_events[i] = t_m[i], x_m[i], y_m[i]
    return viewer.add_points(mito_events, name = "Mitosis events", symbol = "cross", face_color = 'pink')

def plot_target_track(cell_ID):
    target_track = [track for track in all_tracks if track.ID == cell_ID][0]
    target_track_loc = [(target_track.t[i], target_track.x[i]+shift_y, target_track.y[i]+shift_x) for i in range(len(target_track.t))]
    return viewer.add_points(target_track_loc, name = "Track of interest", size = 40, symbol = 'o', face_color = "transparent", edge_color = 'cyan', edge_width = 2)    
    
def plot_stationary_apoptosis_point(cell_ID): ## this function plots apoptotic event and surrounding local environment scope (determined by radius)
    target_track = [track for track in scr_tracks if track.ID == cell_ID][0]
    apop_time, apop_index = find_apoptosis_time(target_track, index = False), find_apoptosis_time(target_track, index = True)  
    apop_event = [(t, target_track.x[apop_index]+shift_y, target_track.y[apop_index]+shift_x) for t in range(len(gfp))] ## marker for apoptosis over all frames
    return viewer.add_points(apop_event, name = "Stastionary apoptosis point", size = 40, symbol = 'o', face_color = "transparent", edge_color = 'cyan', edge_width = 2)

def plot_stationary_apop_radius(cell_ID, radius, delta_t, inner_radius):
    target_track = [track for track in scr_tracks if track.ID == cell_ID][0]
    apop_time, apop_index = find_apoptosis_time(target_track, index = False), find_apoptosis_time(target_track, index = True)  
    apop_event = target_track.t[apop_index], target_track.x[apop_index]+shift_y, target_track.y[apop_index]+shift_x ## with transposed shift, just for the frame of apoptosis
    outer_radial_bin = [tuple(((apop_event[0]+t, apop_event[1]-radius, apop_event[2]-radius), 
                               (apop_event[0]+t, apop_event[1]+radius, apop_event[2]-radius), 
                               (apop_event[0]+t, apop_event[1]+radius, apop_event[2]+radius), 
                               (apop_event[0]+t, apop_event[1]-radius, apop_event[2]+radius))) 
                                for t in range(-abs(delta_t), +abs(delta_t)+1)]
    if inner_radius > 0:
        inner_radial_bin = [tuple(((apop_event[0]+t, apop_event[1]-inner_radius, apop_event[2]-inner_radius), 
                                   (apop_event[0]+t, apop_event[1]+inner_radius, apop_event[2]-inner_radius), 
                                   (apop_event[0]+t, apop_event[1]+inner_radius, apop_event[2]+inner_radius), 
                                   (apop_event[0]+t, apop_event[1]-inner_radius, apop_event[2]+inner_radius))) 
                                    for t in range(-abs(delta_t), +abs(delta_t)+1)]
        return viewer.add_shapes(outer_radial_bin,opacity = 1, shape_type = 'ellipse', face_color = 'transparent', edge_color = 'cyan', edge_width = 5, name = 'Radial environment'), viewer.add_shapes(inner_radial_bin, opacity = 1, shape_type = 'ellipse', face_color = 'transparent', edge_color = 'cyan', edge_width = 5, name = 'Inner Radial environment')
    else:
        return viewer.add_shapes(outer_radial_bin, opacity = 1, shape_type = 'ellipse', face_color = 'transparent', edge_color = 'cyan', edge_width = 5, name = 'Radial environment')

def plot_radius(cell_ID, radius):
    target_track = [track for track in all_tracks if track.ID == cell_ID][0]
    radius_shape = [tuple(((t, target_track.x[i]+shift_y-radius, target_track.y[i]+shift_x-radius), 
                   (t, target_track.x[i]+shift_y+radius, target_track.y[i]+shift_x-radius), 
                   (t, target_track.x[i]+shift_y+radius, target_track.y[i]+shift_x+radius), 
                   (t, target_track.x[i]+shift_y-radius, target_track.y[i]+shift_x+radius))) 
                    for i,t in enumerate(range(target_track.t[0], target_track.t[-1]))]
    return viewer.add_shapes(radius_shape, opacity = 1, shape_type = 'ellipse', face_color = 'transparent', edge_color = 'cyan', edge_width = 5, name = 'Radial environment')
   
def plot_post_track_radius(cell_ID, radius):
    target_track = [track for track in all_tracks if track.ID == cell_ID][0]
    radius_shape = [tuple(((t, target_track.x[-1]+shift_y-radius, target_track.y[-1]+shift_x-radius), 
                   (t, target_track.x[-1]+shift_y+radius, target_track.y[-1]+shift_x-radius), 
                   (t, target_track.x[-1]+shift_y+radius, target_track.y[-1]+shift_x+radius), 
                   (t, target_track.x[-1]+shift_y-radius, target_track.y[-1]+shift_x+radius))) 
                    for i,t in enumerate(range(target_track.t[-1],len(gfp)))]
    return viewer.add_shapes(radius_shape, opacity = 1, shape_type = 'ellipse', face_color = 'transparent', edge_color = 'cyan', edge_width = 5, name = 'Post-apoptosis radial environment')


def plot_fragmented_track(list_of_IDs): ### not using this below as dont think output is correct
    compiled_frag_track_loc = []
    compiled_frag_radius_loc = []
    for cell_ID in list_of_IDs:
        target_track = [track for track in scr_tracks if track.ID == cell_ID][0]
        #plot_radius(target_track)
        #plot_target_track(target_track)
        radius_loc = plot_frag_radius(target_track)
        compiled_frag_radius_loc+= radius_loc
        target_track_loc = plot_frag_target_track(target_track)
        compiled_frag_track_loc += target_track_loc
    return viewer.add_shapes(compiled_frag_radius_loc, opacity = 1, shape_type = 'ellipse', face_color = 'transparent', edge_color = 'cyan', edge_width = 5, name = 'Radial environment'), viewer.add_points(compiled_frag_track_loc, name = "Track of interest", size = 40, symbol = 'o', face_color = "transparent", edge_color = 'cyan', edge_width = 2)
    
def plot_frag_target_track(target_track):
    if target_track.ID == 17:
        target_track_loc = [(target_track.t[i], target_track.x[i]+shift_y, target_track.y[i]+shift_x) for i in range(len(target_track.t))]
        return target_track_loc #viewer.add_points(target_track_loc, name = "Track of interest", size = 40, symbol = 'o', face_color = "transparent", edge_color = 'cyan', edge_width = 2)    
    else:
        target_track_loc = [(target_track.t[i], target_track.x[i]+shift_y, target_track.y[i]+shift_x) for i in range(len(target_track.t)) if target_track.t[i]> 742]
        return target_track_loc#viewer.add_points(target_track_loc, name = "Track of interest", size = 40, symbol = 'o', face_color = "transparent", edge_color = 'cyan', edge_width = 2)    

def plot_frag_radius(target_track):
    if target_track.ID ==17:### this if condition is to avoid double plotting radii as fragmented tracks exist at same time
        radius_shape = [tuple(((t, target_track.x[i]+shift_y-radius, target_track.y[i]+shift_x-radius), 
                       (t, target_track.x[i]+shift_y+radius, target_track.y[i]+shift_x-radius), 
                       (t, target_track.x[i]+shift_y+radius, target_track.y[i]+shift_x+radius), 
                       (t, target_track.x[i]+shift_y-radius, target_track.y[i]+shift_x+radius))) 
                        for i,t in enumerate(range(target_track.t[0], target_track.t[-1]))]
        return radius_shape
    else:
        radius_shape = [tuple(((t, target_track.x[i]+shift_y-radius, target_track.y[i]+shift_x-radius), 
                       (t, target_track.x[i]+shift_y+radius, target_track.y[i]+shift_x-radius), 
                       (t, target_track.x[i]+shift_y+radius, target_track.y[i]+shift_x+radius), 
                       (t, target_track.x[i]+shift_y-radius, target_track.y[i]+shift_x+radius))) 
                        for i,t in enumerate(range(target_track.t[0], target_track.t[-1])) if t>741]
        return radius_shape
    

# false cell of interest to plot relabelled track post apoptosis

In [71]:
false_target_track = [scr_track for scr_track in scr_tracks if scr_track.ID == 47][0]

In [11]:
radius = 500

In [12]:
delta_t = 100

# fixing fragmented display

In [20]:
compiled_frag_track_loc = []
compiled_frag_radius_loc = []
for i in [17,42,47]:
    target_track = [track for track in scr_tracks if track.ID == cell_ID][0]
    #plot_radius(target_track)
    #plot_target_track(target_track)
    radius_loc = plot_frag_radius(target_track)
    compiled_frag_radius_loc+= radius_loc
    target_track_loc = plot_frag_target_track(target_track)
    compiled_frag_track_loc += target_track_loc

In [21]:
compiled_frag_track_loc[-1]

(742, 621.9642944335938, 1179.050048828125)

In [22]:
target_track

,ID,t,x,y,z,parent,root,state,generation,dummy
0,17,354,592.089050,1121.349365,0.000000,2,2,3,1,False
1,17,355,599.770813,1122.750000,0.000000,2,2,3,1,False
2,17,356,597.294983,1122.762573,0.000000,2,2,3,1,False
3,17,357,595.140930,1121.939575,0.000000,2,2,3,1,False
4,17,358,596.659119,1121.590942,0.000000,2,2,3,1,False
5,17,359,598.070312,1123.187500,0.000000,2,2,3,1,False
6,17,360,601.812317,1124.481201,0.000000,2,2,3,1,False
7,17,361,606.079834,1126.327026,0.000000,2,2,3,1,False
8,17,362,608.422485,1126.737915,0.000000,2,2,0,1,False
9,17,363,610.502502,1127.165894,0.000000,2,2,0,1,False


# trying movie output

In [13]:
from naparimovie import Movie
#

### Launch napari to check against raw data 

In [32]:
with napari.gui_qt():
    viewer = napari.Viewer()
    
    #viewer.add_image(bf, name = "brightfield")
    viewer.add_image(gfp, name="gfp", contrast_limits = (30, 180), blending = "additive", colormap = "green")
    viewer.add_image(rfp, name="rfp", contrast_limits = (4, 20), blending = "additive", colormap = "magenta")

    viewer.add_tracks(wt_data)
    viewer.add_tracks(scr_data)
    
    #movie = Movie(myviewer=viewer)
    
    ### only use additional annotation if necessary, otherwise itll take a while to load
    #plot_wt_target_track(wt_target_track)
    #plot_target_track(target_track)
    #plot_target_track(false_target_track)
    #plot_radius(false_target_track)
    #plot_post_apoptosis()
    #plot_post_apop_radius()
    #plot_radial_bin() ## the larger delta_t, the longer it will take to load
    #plot_mitoses() ## utilises two functions above (wt_tracks_in_radius) to plot apop and mito 
    #plot_apoptosis(ever_present = True)
    #plot_radius(target_track)
    
    
    ### initial analysis
    plot_stationary_apoptosis_point(17)
    plot_stationary_apop_radius(17, 800, 20, 0)
    ## updated follow analysis 
    #plot_fragmented_track([17,42,47])
#     plot_radius(-17, 500)
#     plot_target_track(-17)
    viewer.dims.set_point(0, (apop_time + delta_t)) ## open viewer at the timepoint of interest